In [ ]:
import pandas as pd

In [ ]:

Avatar=pd.read_csv("./csvs/Avatar"),
Partida=pd.read_csv("./csvs/Partida"),
Deseado=pd.read_csv("./csvs/Deseado"),
Biblioteca=pd.read_csv("./csvs/Biblioteca"),
Usuario=pd.read_csv("./csvs/Usuario"),
Productora=pd.read_csv("./csvs/Productora"),
Saga=pd.read_csv("./csvs/Saga"),
Juego=pd.read_csv("./csvs/Juego")
    

# Generacion de los jsons

### Partidas de los jugadores

In [ ]:
# convert date columns to unix timestamps
date_columns = ['FechaInicio', 'FechaFin']

for df in [Partida]:
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).astype(int)# convert to unix timestamp in seconds

# create nested json structure
productoras = []

for productora_id, productora_group in Productora.groupby('IdProductora'):
    productora_data = {
        'nombre_productora': productora_group['Nombre'].iloc[0],
        'sagas': []
    }
    
    for saga_id, saga_group in Saga[Saga['IdProductora'] == productora_id].groupby('IdSaga'):
        saga_data = {
            'nombre_saga': saga_group['Nombre'].iloc[0],
            'juegos': []
        }
        
        for juego_id, juego_group in Juego[Juego['IdSaga'] == saga_id].groupby('IdJuego'):
            juego_data = {
                'titulo_juego': juego_group['Titulo'].iloc[0],
                'partidas': []
            }
            
            for _,partida_group in Partida[Partida['IdJuego'] == juego_id].groupby('IdJuego'):
                
                for i in range(partida_group.shape[0]):
                
                    idUsuario = partida_group['IdUsuario'].iloc[i]
                    fecha_ini = partida_group['FechaInicio'].iloc[i]
                    fecha_fin = partida_group['FechaFin'].iloc[i]
                
                    partida_data = {
                        'fechaini': fecha_ini,
                        'fechafin': fecha_fin,
                        'duracion': fecha_ini - fecha_fin,
                        'usuario': 
                        {
                            'avatar':Avatar[Avatar['IdUsuario'] == idUsuario]['Nombre'].iloc[0],
                            'username':Usuario[Usuario['IdUsuario'] == idUsuario]['NombreUsuario'].iloc[0]
                        }
                    }
                    juego_data['partidas'].append(partida_data)
            
            saga_data['juegos'].append(juego_data)
        
        productora_data['sagas'].append(saga_data)
    
    productoras.append(productora_data)

# convert to json
json_data = productoras

### Compras

In [ ]:
# convert date columns to unix timestamps
date_columns = ['FechaInicio', 'FechaFin']

for df in [Partida]:
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).astype(int) // 10**9  # convert to unix timestamp in seconds

# create nested json structure
productoras = []

for productora_id, productora_group in Productora.groupby('IdProductora'):
    productora_data = {
        'nombre_productora': productora_group['Nombre'].iloc[0],
        'sagas': []
    }
    
    for saga_id, saga_group in Saga[Saga['IdProductora'] == productora_id].groupby('IdSaga'):
        saga_data = {
            'nombre_saga': saga_group['Nombre'].iloc[0],
            'juegos': []
        }
        
        for juego_id, juego_group in Juego[Juego['IdSaga'] == saga_id].groupby('IdJuego'):
            juego_data = {
                'titulo_juego': juego_group['Titulo'].iloc[0],
                'ventas': []
            }
            
            for _,biblioteca_group in Biblioteca[Biblioteca['IdJuego'] == juego_id].groupby('IdJuego'):
                
                for i in range(biblioteca_group.shape[0]):
                
                    idUsuario = biblioteca_group['IdUsuario'].iloc[i]
                    idJuego = biblioteca_group['IdJuego'].iloc[i]
                    
                    ventas_data = {
                        'fecha_compra': biblioteca_group['FechaAdquisicion'].iloc[i],
                        'modo_pago':biblioteca_group['ModoAdquisicion'].iloc[i],
                        'precio':biblioteca_group['PrecioCompra'].iloc[i],
                        'descuento':biblioteca_group['Descuento'].iloc[i],
                        'juego':Juego[Juego['IdJuego'] == idJuego]['Titulo'].iloc[0],
                        'usuario': Usuario[Usuario['IdUsuario'] == idUsuario]['NombreUsuario'].iloc[0]
                        
                    }
                    juego_data['ventas'].append(ventas_data)
            
            saga_data['juegos'].append(juego_data)
        
        productora_data['sagas'].append(saga_data)
    
    productoras.append(productora_data)

# convert to json
json_data_compras = productoras

### Guardado

In [ ]:
with open('./jsons/jsosns_partidas.json', 'w+') as file:json.dump(json_data, fp = file,separators=(', ', ': '), ensure_ascii=False,cls=NumpyEncoder)

In [ ]:
with open('./jsons/jsosns_compras.json', 'w+') as file:json.dump(json_data_compras, fp = file,separators=(', ', ': '), ensure_ascii=False,cls=NumpyEncoder)